In [61]:
%matplotlib inline
import json
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [13]:
try:
    rj2 = pd.read_csv('../../datasets/eleicao_rj/limpo.csv', index_col=0)
except FileNotFoundError:
    df = pd.read_csv(
        '../../datasets/eleicao_rj/votacao_secao_2018_RJ.csv',
        sep=';',
        encoding='Latin1')
    rj = df.query('DS_CARGO == "GOVERNADOR" and NM_UE == "RIO DE JANEIRO"')
    del df
    rj2 = rj.drop([
        'DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
        'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO',
        'DT_ELEICAO', 'TP_ABRANGENCIA', 'CD_MUNICIPIO', 'SG_UF', 'SG_UE',
        'NM_UE', 'CD_CARGO', 'DS_CARGO'
    ],
                  axis=1)
    rj2.columns = [
        'municipio', 'zona', 'secao', 'nr_candidato', 'candidato', 'votos'
    ]
    rj2.to_csv('../../datasets/eleicao_rj/limpo.csv')
    del rj
    rj2.loc[:, 'turno'] = 1

In [39]:
rj = pd.read_csv(
    '../../datasets/eleicao_rj/resultado_eleicao.csv',
    encoding='Latin1',
    sep=';')
rj = rj.loc[:, [
    'Nome do candidato', 'Zona', 'Turno', 'Votos nominais do candidato'
]]
rj.columns = ['candidato', 'zona', 'turno', 'votos']

votos
zona candidato             turno       
4    EDUARDO DA COSTA PAES 2      59716
     WILSON JOSÉ WITZEL    2      22762
5    EDUARDO DA COSTA PAES 2      53070
     WILSON JOSÉ WITZEL    2      25835
7    EDUARDO DA COSTA PAES 2      49610
     WILSON JOSÉ WITZEL    2      25416
8    EDUARDO DA COSTA PAES 2      31443
     WILSON JOSÉ WITZEL    2      25278
9    EDUARDO DA COSTA PAES 2      47370
     WILSON JOSÉ WITZEL    2      35751
10   EDUARDO DA COSTA PAES 2      29584
     WILSON JOSÉ WITZEL    2      26836
14   EDUARDO DA COSTA PAES 2      32073
     WILSON JOSÉ WITZEL    2      27799
16   EDUARDO DA COSTA PAES 2      56481
     WILSON JOSÉ WITZEL    2      23211
17   EDUARDO DA COSTA PAES 2      58836
     WILSON JOSÉ WITZEL    2      18920
21   EDUARDO DA COSTA PAES 2      27742
     WILSON JOSÉ WITZEL    2      28042
22   EDUARDO DA COSTA PAES 2      33041
     WILSON JOSÉ WITZEL    2      40744
23   EDUARDO DA COSTA PAES 2      25140
     WILSON JOSÉ WITZEL    2      31098
24   EDUARDO DA COSTA PAES 2      25974
     WILSON JOSÉ WITZEL    2      38891
25   EDUARDO DA COSTA PAES 2      23438
     WILSON JOSÉ WITZEL    2      28448
26   EDUARDO DA COSTA PAES 2      13048
     WILSON JOSÉ WITZEL    2      25346
...                                 ...
222  EDUARDO DA COSTA PAES 2      14036
     WILSON JOSÉ WITZEL    2      37391
225  EDUARDO DA COSTA PAES 2      10218
     WILSON JOSÉ WITZEL    2      25941
229  EDUARDO DA COSTA PAES 2      46300
     WILSON JOSÉ WITZEL    2      26826
230  EDUARDO DA COSTA PAES 2      14784
     WILSON JOSÉ WITZEL    2      23790
233  EDUARDO DA COSTA PAES 2      29471
     WILSON JOSÉ WITZEL    2      42205
234  EDUARDO DA COSTA PAES 2      23506
     WILSON JOSÉ WITZEL    2      31998
238  EDUARDO DA COSTA PAES 2      27111
     WILSON JOSÉ WITZEL    2      45258
241  EDUARDO DA COSTA PAES 2      15413
     WILSON JOSÉ WITZEL    2      25566
242  EDUARDO DA COSTA PAES 2      16859
     WILSON JOSÉ WITZEL    2      27904
243  EDUARDO DA COSTA PAES 2      18966
     WILSON JOSÉ WITZEL    2      25367
245  EDUARDO DA COSTA PAES 2      24271
     WILSON JOSÉ WITZEL    2      43949
246  EDUARDO DA COSTA PAES 2      24030
     WILSON JOSÉ WITZEL    2      40206
254  EDUARDO DA COSTA PAES 2      11102
     WILSON JOSÉ WITZEL    2      30366
255  EDUARDO DA COSTA PAES 2       6541
     WILSON JOSÉ WITZEL    2      11090
256  EDUARDO DA COSTA PAES 2       9580
     WILSON JOSÉ WITZEL    2      34534

[330 rows x 1 columns]

## Limpeza dos dados das zonas eleitorais

In [80]:
end_zonas = pd.read_csv(
    '../../datasets/eleicao_rj/lista_zonas_eleitorais.csv',
    encoding='Latin1',
    index_col=0)
end_zonas = end_zonas.rename(columns={'endereco': 'rua',
                                     'nome_municipio': 'municipio'})
end_zonas = end_zonas.drop(['sigla_uf', 'cep', 'sigla_uf.1'], axis=1)
end_zonas['endereco'] = end_zonas.apply(lambda x: ', '.join(x), axis=1)
colunas = ['rua', 'bairro', 'municipio', 'endereco']
for c in colunas:
    end_zonas.loc[:, c] = end_zonas.loc[:, c].apply(lambda x: x.lower())

Vamos utilizar o serviço do google para geocodificar os endereços das zonas caso não exista o arquivo pronto.

In [81]:
erros = []
try:
    end_zonas = pd.read_csv(
        '../../datasets/eleicao_rj/zonas_eleitorais_proc.csv', index_col=0)
except FileNotFoundError:
    dados_json = []
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    bounds = '-23.371469,-44.917957|-20.784440,-40.504246'
    with open('gkey.txt') as f:
        g_key = f.read()
    for z, v in end_zonas.iterrows():
        end = v["endereco"].replace(' ', '+')
        response = requests.get(url, {
            'bounds': bounds,
            'key': g_key,
            'address': end
        })
        try:
            coords = json.loads(
                response.text)['results'][0]['geometry']['location']
            end_zonas.loc[z, 'lat'] = coords['lat']
            end_zonas.loc[z, 'lng'] = coords['lng']
            dados_json.append(response.json())
        except:
            erros.append(z)
            print(f'Não achou zona {z}')
        time.sleep(0.05)
    end_zonas.loc[60, 'lat'] = -21.954952
    end_zonas.loc[60, 'lng'] = -42.136810
    end_zonas.loc[196, 'lat'] = -22.151659
    end_zonas.loc[196, 'lng'] = -42.926066
    end_zonas.index.name = 'zona'
    end_zonas.to_csv('../../datasets/eleicao_rj/zonas_eleitorais_proc.csv')

Não achou zona 60


## Limpeza dos dados dos votos
Vamos criar dois datasets diferentes, um contendo o percentual dos votos de cada zona eleitoral foi para cada candidato e o outro contendo a representatividade de votos de cada zona eleitoral em relação ao total de cada candidato.

In [105]:
votos_zona.loc[z].loc[t]

44114

In [113]:
for (zo, c, tu), v in (votos.query(f'zona == {z} & turno == {t}')/votos_zona.loc[z].loc[t]).iterrows():
    print(f'zona: {zo}, candidato: {c}, turno: {tu}, votos: {v.votos}')

zona: 256, candidato: EDUARDO DA COSTA PAES, turno: 2, votos: 0.21716461894183253
zona: 256, candidato: WILSON JOSÉ WITZEL, turno: 2, votos: 0.7828353810581675


In [82]:
votos = pd.concat([
    rj2.loc[:, ['zona', 'candidato', 'votos', 'turno']].set_index('zona'),
    rj.set_index('zona')
])
votos_zona = np.sum(votos.groupby(['zona', 'turno']))['votos']
votos_candidato = np.sum(votos.groupby('candidato')).loc[:, 'votos']
votos = np.sum(votos.groupby(['zona', 'candidato', 'turno']))
votos_perc_zona = pd.DataFrame(columns=list(votos.index.levels[1]))
votos_perc_cand = pd.DataFrame(columns=list(votos.index.levels[1]))
for z, t in votos_zona.index.values:
    votos.query(f'zona == {z} & turno == {t}')/votos_zona.loc[z].loc[t]
    for i, v in (votos.loc[z] / votos_zona[z]).iterrows():
        votos_perc_zona.loc[z, i] = round(v[0] * 100, 2)
    for i, v in (votos.loc[z] / votos_candidato).iterrows():
        votos_perc_cand.loc[z, i] = round(v[0] * 100, 2)
votos_perc_cand.index.name = 'zona'
votos_perc_zona.index.name = 'zona'

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [1] of <class 'int'>

In [ ]:
votos_perc_cand.to_csv('../../datasets/eleicao_rj/votos_perc_cand.csv')
votos_perc_zona.to_csv('../../datasets/eleicao_rj/votos_perc_zona.csv')

## Enriquecendo os dados

A partir dos dados do IDH Municipal, agregaremos esses dados na nossa análise de forma para enriquecermos a nossa análise.

In [ ]:
idhm = pd.read_excel(
    '../../datasets/IDHM/IDHM_Municipios_RJ.xlsx', index_col=0)
idhm.index.name = 'cod'
idhm.columns = ['municipio', 'idhm']
idhm.head(5)

In [78]:
colunas = ['rua', 'bairro', 'nome_municipio', 'endereco']
for c in colunas:
    end_zonas.loc[:, c] = end_zonas.loc[:, c].apply(lambda x: x.lower())

In [79]:
end_zonas

,rua,bairro,nome_municipio,endereco,lat,lng
zona,,,,,,
4,rua jardim botânico 1060,jardim botânico,rio de janeiro,"rua jardim botânico 1060, jardim botânico, rio...",-22.972010,-43.224547
5,"rua miguel lemos, 97",copacabana,rio de janeiro,"rua miguel lemos, 97, copacabana, rio de janeiro",-22.977688,-43.192630
7,rua antonio basilio 76,tijuca,rio de janeiro,"rua antonio basilio 76, tijuca, rio de janeiro",-22.924920,-43.235859
8,"avenida dom hélder câmara, 4175",del castilho,rio de janeiro,"avenida dom hélder câmara, 4175, del castilho,...",-22.881511,-43.274248
9,av ayrton senna 2001 bloco c,barra da tijuca,rio de janeiro,"av ayrton senna 2001 bloco c, barra da tijuca,...",-22.992552,-43.367683
10,"rua assis carneiro, 436",piedade,rio de janeiro,"rua assis carneiro, 436, piedade, rio de janeiro",-22.893737,-43.312016
14,"rua getulio, 127",todos os santos,rio de janeiro,"rua getulio, 127, todos os santos, rio de janeiro",-22.895333,-43.280462
16,rua conde de baependi n 40,laranjeiras,rio de janeiro,"rua conde de baependi n 40, laranjeiras, rio d...",-22.932624,-43.179049
17,rua jardim botânico 1060,jardim botânico,rio de janeiro,"rua jardim botânico 1060, jardim botânico, rio...",-22.972010,-43.224547
